# 📊 PLS 120 Week 10: ANOVA and Linear Regression
**Applied Statistics in Agricultural Sciences**

**Binder Developer:** Mohammadreza Narimani  
**Lab Content Developer:** Parastoo Farajpoor  
**Date:** December 3, 2024

---

## 🎯 Learning Objectives

By the end of this lab, you will be able to:
- Build ANOVA tables from scratch
- Calculate sum of squares, degrees of freedom, and F-values manually
- Perform linear regression analysis
- Interpret ANOVA and regression results
- Use R functions for statistical modeling

## 📦 Load Required Packages

In [ ]:
# Load required packages
library(ggplot2)
library(dplyr)

print("📊 Packages loaded successfully!")

---

## 🌸 ANOVA Analysis

First, we will work on building an ANOVA table from the ground up. There are simpler ways to do an ANOVA, however, those require some knowledge about linear models, which we will focus on shortly. We will use the iris data to perform an ANOVA to determine whether there is a difference between petal width.

**Null hypothesis:** There is no effect of species on petal width (petal width between groups is not significantly different)

**Alternative hypothesis:** There is an effect of species on petal width (petal width between groups is significantly different)

In [ ]:
# Load the iris dataset
flower <- iris

# Create boxplots to visualize the data
ggplot(flower, aes(x = Species, y = Petal.Width, fill = Species)) + 
  geom_boxplot() +
  labs(title = "Petal Width by Species", 
       x = "Species", 
       y = "Petal Width (cm)")

# Alternative boxplot using base R
boxplot(Petal.Width ~ Species, data = flower, 
        main = "Petal Width by Species", 
        xlab = "Species", 
        ylab = "Petal Width (cm)", 
        col = c("lightblue", "lightgreen", "lightyellow"))

In [ ]:
# Check data structure
str(flower)

# When using R, it's important to check whether our data is formatting correctly. 
# R will default to certain data formats depending on what the data looks like. 
# We want our numerical data to be categorized as "num" and the species to be categorized as "Factor".

## 📐 Sum of Squares Calculations

The first step we will need to do is calculate the overall mean for petal width for the population, then the individual means for each of the species.

In [ ]:
# Calculate overall mean
overall_mean <- mean(flower$Petal.Width)
cat("Overall mean petal width:", overall_mean, "cm\n")

# Add this overall mean to the dataframe as a new column
flower$Overall_Avg <- overall_mean
head(flower)

In [ ]:
# Method A: Create different dataframes for each species
setosa <- flower %>% filter(Species == "setosa")
versicolor <- flower %>% filter(Species == "versicolor")
virginica <- flower %>% filter(Species == "virginica")

setosa_mean <- mean(setosa$Petal.Width)
setosa_count <- nrow(setosa)

versicolor_mean <- mean(versicolor$Petal.Width)
versicolor_count <- nrow(versicolor)

virginica_mean <- mean(virginica$Petal.Width)
virginica_count <- nrow(virginica)

cat("Setosa mean:", setosa_mean, "(n =", setosa_count, ")\n")
cat("Versicolor mean:", versicolor_mean, "(n =", versicolor_count, ")\n")
cat("Virginica mean:", virginica_mean, "(n =", virginica_count, ")\n")

In [ ]:
# Method B: Direct calculation without separate dataframes
setosa_mean <- mean(flower[flower$Species == "setosa", ]$Petal.Width)
setosa_count <- nrow(flower[flower$Species == "setosa", ])

versicolor_mean <- mean(flower[flower$Species == "versicolor", ]$Petal.Width)
versicolor_count <- nrow(flower[flower$Species == "versicolor", ])

virginica_mean <- mean(flower[flower$Species == "virginica", ]$Petal.Width)
virginica_count <- nrow(flower[flower$Species == "virginica", ])

# Create a summary table
mean_table <- data.frame(
  Species = c("setosa", "versicolor", "virginica"),
  Mean_Petal_Width = c(setosa_mean, versicolor_mean, virginica_mean),
  Count = c(setosa_count, versicolor_count, virginica_count)
)
print(mean_table)

In [ ]:
# Alternative method using dplyr
summary_stats <- flower %>%
  group_by(Species) %>%
  summarise(
    Mean_Petal_Width = mean(Petal.Width),
    Count = n(),
    .groups = 'drop'
  )
print(summary_stats)

Once we have the total average and the average for individual species, we can calculate the sum of squares (between groups, within groups and total SS)

In [ ]:
## 1. SSB (Sum of Squares Between): 
# Calculated by taking the sum of the squared differences between each group's mean 
# and the overall mean, weighted by the group size. This reflects how much the group 
# means differ from the overall mean.

SSB <- nrow(setosa) * (setosa_mean - overall_mean)^2 +
       nrow(versicolor) * (versicolor_mean - overall_mean)^2 +
       nrow(virginica) * (virginica_mean - overall_mean)^2

cat("Sum of Squares Between (SSB):", SSB, "\n")

In [ ]:
## 2. SST (Total Sum of Squares): 
# Reflects all deviations from the overall mean, combining within-group and 
# between-group variability. It measures the total variance in the response variable.

SST <- sum((flower$Petal.Width - overall_mean)^2)
cat("Total Sum of Squares (SST):", SST, "\n")

In [ ]:
## 3. SSE (Sum of Squares Within/Error): 
# The sum of squared deviations of each observation from its group mean. 
# Since SST = SSB + SSE, we can use algebra to determine SSE.

SSE <- SST - SSB
cat("Sum of Squares Error (SSE):", SSE, "\n")

# Alternative calculation for SSE
SSE_alt <- sum((setosa$Petal.Width - setosa_mean)^2) + 
           sum((versicolor$Petal.Width - versicolor_mean)^2) + 
           sum((virginica$Petal.Width - virginica_mean)^2)
cat("SSE (alternative calculation):", SSE_alt, "\n")

# Verify SST = SSB + SSE
cat("Verification: SSB + SSE =", SSB + SSE, "should equal SST =", SST, "\n")

## 🔢 Degrees of Freedom

Now we will calculate the degrees of freedom. We will start by determining our k and N values. From there, we can easily calculate the individual degrees of freedom.

In [ ]:
# k is the number of "groups" we are examining. 
# In our case, we are examining three different species. Therefore k=3.
k <- 3

# N is the total number of observations.
N <- nrow(flower)

# The first degree of freedom is between groups DoF. This measurement is just k-1.
df1 <- k - 1

# To calculate the degree of freedom for the error, we need to subtract 
# the total number of observations by the number of different groups, or N-k.
df2 <- N - k

# The total degrees of freedom is just the number of all observations minus 1, or N-1.
df3 <- N - 1

cat("Number of groups (k):", k, "\n")
cat("Total observations (N):", N, "\n")
cat("Degrees of freedom between groups (df1):", df1, "\n")
cat("Degrees of freedom within groups (df2):", df2, "\n")
cat("Total degrees of freedom (df3):", df3, "\n")

## 📊 Mean Squares

Mean square is the Sum of squares divided by degrees of freedom.

In [ ]:
# Mean squares between groups (MSB)
MSB <- SSB / df1
cat("Mean Square Between (MSB):", MSB, "\n")

# Mean squares within groups/error (MSE)
MSE <- SSE / df2
cat("Mean Square Error (MSE):", MSE, "\n")

## 🔥 F Value

Finally, using the two mean square values, we will calculate F value. In general, the higher the F value, the more likely you are to reject the null hypothesis (the null hypothesis being the mean of groups are not significantly different). However, the F statistic alone does not determine this. You must examine the F value relative to the F distribution and its associated p-value.

In [ ]:
# Calculate F value
F_value <- MSB / MSE
cat("F-statistic:", F_value, "\n")

# This statistic is a measure of how much the species mean petal width 
# differs relative to the variation within species.

# Create ANOVA summary table
anova_table <- data.frame(
  Source = c("Between Groups", "Within Groups", "Total"),
  SS = c(SSB, SSE, SST),
  df = c(df1, df2, df3),
  MS = c(MSB, MSE, NA),
  F_value = c(F_value, NA, NA)
)
print(anova_table)

To determine if the observed F value is large enough to reject the null hypothesis (that all group means are equal), compare it to a critical F value from the F-distribution. The critical F value depends on the degrees of freedom of the numerator (between groups) and the denominator (within groups) and the significance level (commonly α=0.05).

In practical applications, especially in R, you often don't manually compare the F value to a critical value because R provides a p-value. The p-value tells you the probability of observing an F value as extreme as, or more extreme than, the value calculated from your data if the null hypothesis is true.

In [ ]:
# The simple way of doing this requires you to build a linear model. 
# First, you make a model that describes how a factor or group or a variable (species) 
# will influence a response variable (petal width). 
# You build the model using the lm() function. 
# Then, you put that model in anova() function, and R will generate the table for you.

# lm(response variable ~ independent variable)
model <- lm(Petal.Width ~ Species, flower)
anova_result <- anova(model)
print(anova_result)

cat("\nThe p-value is significant, indicating strong evidence against the null hypothesis\n")
cat("that all species have the same mean petal width. In other words, it suggests\n")
cat("that the observed difference in means among the iris species is highly statistically significant.\n")

---

## 📈 Linear Regression

Linear Regression is creating a linear model based on the relationship between 2 or more variables. Usually, we want to predict the dependent variable (y or response variable) based on one or more independent variables (x or treatments).

In [ ]:
# Creating a linear model using Species (independent variable) to predict Petal Width (dependent variable). 
# This is only when you want to compare the effect of your groups in your dependent variables. 
# This model doesn't give a "Line" as a model.
model_1 <- lm(Petal.Width ~ Species, flower)
summary(model_1)

In [ ]:
# Calculate correlation matrix for all numeric variables
correlation_matrix <- cor(iris[, 1:4])
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
# Creating a linear model using Petal Length (independent variable) to predict Petal Width (dependent variable)
model_2 <- lm(Petal.Width ~ Petal.Length, flower)
summary(model_2)

# Plot the relationship
ggplot(flower, aes(x = Petal.Length, y = Petal.Width)) +
  geom_point(alpha = 0.5) + # This alpha adjusts the transparency of points
  geom_smooth(method = "lm", se = FALSE, color = "blue") + # se=FALSE removes confidence interval shade
  labs(title = "Petal Width vs Petal Length",
       x = "Petal Length (cm)",
       y = "Petal Width (cm)")

In [ ]:
# Creating a linear model using Sepal Width (independent variable) to predict Petal Width (dependent variable)
model_3 <- lm(Petal.Width ~ Sepal.Width, flower)
summary(model_3)

# Plot the relationship
ggplot(flower, aes(x = Sepal.Width, y = Petal.Width)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE, color = "blue") +
  labs(title = "Petal Width vs Sepal Width",
       x = "Sepal Width (cm)",
       y = "Petal Width (cm)")

---

## 📋 Summary

In this lab, we learned:

1. **ANOVA Analysis:**
   - How to build ANOVA tables from scratch
   - Calculate sum of squares (SSB, SSE, SST)
   - Determine degrees of freedom
   - Compute mean squares and F-statistics
   - Use R's built-in ANOVA functions

2. **Linear Regression:**
   - Create linear models with categorical and continuous predictors
   - Interpret regression output
   - Visualize relationships between variables
   - Calculate correlation matrices

3. **Statistical Interpretation:**
   - Understanding p-values and significance
   - Comparing manual calculations with R functions
   - Making conclusions based on statistical evidence

---

## 📧 Questions or Need Help?

If you have questions about this lab or need help with R programming, please contact:

**Mohammadreza Narimani**  
📧 mnarimani@ucdavis.edu  
🏫 Department of Biological and Agricultural Engineering, UC Davis  
⏰ Office Hours: Thursdays 10 AM - 12 PM (Zoom)  
🔗 [Join Zoom Office Hours](https://ucdavis.zoom.us/j/99533096447)

---

**Great work completing Week 10!** 🎉📊